In [1]:
!pip install normflows

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 5.3 MB/s 
  Created wheel for normflows: filename=normflows-1.4-py2.py3-none-any.whl size=67650 sha256=eafac1fd9820ba42be8f514b0ea3839baf0b4d64498e02b8cbfa95285f52fd47
  Stored in directory: /root/.cache/pip/wheels/26/8b/1a/bd25def11bba94f0f7801cd67db0f22f8b9aab7b29ee99b9fb
Successfully built normflows


In [2]:
import pandas as pd
import numpy as np
import normflows as nf
import torch
import random
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import math

In [3]:
def zscore(x,mean,std):
  return (x-mean)/std

In [4]:
#Load a CSV into a Pandas dataframe
df = pd.read_csv('ionosphere.csv',sep=',',header=0,engine='python')

In [5]:
df

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


In [6]:
df.drop(['1'],axis=1,inplace=True)

In [7]:
df

,0,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,1,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,0.89391,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,0.96510,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,0.94124,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,0.89724,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


In [8]:
df[df.columns[-1]].replace(['g','b'],[1,0],inplace=True)

In [9]:
df

,0,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,1,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
1,1,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,0.89391,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,1
347,1,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,0.96510,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,1
348,1,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,0.94124,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,1
349,1,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,0.89724,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,1


In [10]:
def get_flow(input_size,output_size,n_layers,n_units,n_blocks):
  flows = []
  for i in range(n_blocks):
    net = nf.nets.LipschitzMLP([input_size*output_size] + [n_units] * (n_layers - 1) + [input_size*output_size],
                               init_zeros=True, lipschitz_const=0.9)
    flows += [nf.flows.Residual(net, reduce_memory=True)]
  z = nf.distributions.DiagGaussian(input_size*output_size, trainable=False)
    
  # Construct flow model
  return nf.NormalizingFlow(q0=z, flows=flows)

In [11]:
class ScaleMixtureGaussian(object):
    def __init__(self, pi, sigma1, sigma2):
        super().__init__()
        self.pi = pi
        self.sigma1 = sigma1
        self.sigma2 = sigma2
        self.gaussian1 = torch.distributions.Normal(0,sigma1)
        self.gaussian2 = torch.distributions.Normal(0,sigma2)
    
    def sample(self,shape,n_samples=1):
        u = random.uniform(0.0,1.0)
        samples = []
        for i in range(n_samples):
          if u <self.pi:
            samples.append(self.gaussian1.rsample(shape))
          else:
            samples.append(self.gaussian2.rsample(shape))
        return torch.stack(samples)

    def log_prob(self, input):
        prob1 = torch.exp(self.gaussian1.log_prob(input))
        prob2 = torch.exp(self.gaussian2.log_prob(input))
        return (torch.log(self.pi * prob1 + (1-self.pi) * prob2)).sum()

In [12]:
class logistic_nf(torch.nn.Module):
    def __init__(self, inputSize):
        super(logistic_nf, self).__init__()
        PI = 0.5
        SIGMA_1 = torch.FloatTensor([math.exp(-0)])
        SIGMA_2 = torch.FloatTensor([math.exp(-6)])
        self.prior = ScaleMixtureGaussian(PI,SIGMA_1,SIGMA_2)
        self.weights = get_flow(inputSize,2,3,20,1)
        self.input_size = inputSize
    def forward(self, x,stds=None):
        sample, log_q = self.weights.sample()
        out = F.softmax(F.linear(x,torch.reshape(sample,[2,self.input_size])))
        return out, log_q, self.prior.log_prob(sample)


In [13]:
not_test, test = train_test_split(df,test_size = 0.25,random_state=42)
train, validation = train_test_split(not_test,test_size = 0.33,random_state=42)

In [14]:
train_mean = train.mean(axis=0)
train_std = train.std(axis=0,ddof=0)
for c in train.columns[:-1]:
  train[c] = zscore(train[c],train_mean[c],train_std[c])
for c in validation.columns[:-1]:
  validation[c] = zscore(validation[c],train_mean[c],train_std[c])
for c in test.columns[:-1]:
  test[c] = zscore(test[c],train_mean[c],train_std[c]) 

In [15]:
x_train_numpy = train[df.columns[:-1]].to_numpy()
y_train_numpy = train[df.columns[-1]].to_numpy(dtype=np.uint8)
x_train = torch.from_numpy(x_train_numpy).float()
y_train = torch.from_numpy(y_train_numpy)
x_validation_numpy = validation[df.columns[:-1]].to_numpy()
y_validation_numpy = validation[df.columns[-1]].to_numpy(dtype=np.uint8)
x_validation = torch.from_numpy(x_validation_numpy).float()
y_validation = torch.from_numpy(y_validation_numpy)

In [16]:
from normflows.core import NormalizingFlow
from torch.distributions.utils import logits_to_probs
model = logistic_nf(x_train_numpy.shape[1])
opt = torch.optim.Adam(params=model.weights.parameters(),lr=0.001)

In [17]:
n_mc_samples = 20 
ps = []
for m in range(n_mc_samples):
    p_tmp, _, _ = model(x_validation)
    ps.append(p_tmp.detach().numpy())
ps = np.stack(ps)
p = ps.mean(axis=0)
p_var = ps.var(axis=0).sum(axis=-1)
acc = (p.argmax(axis=1) == y_validation.detach().numpy()).mean()
print('Acc.')
print(acc)
print('Var.')
print(p_var)

Acc.
0.6436781609195402
Var.
[0.37733358 0.4344709  0.38044545 0.38553792 0.43856943 0.28460795
 0.18889624 0.46475723 0.276199   0.27421892 0.40521023 0.46460867
 0.4265744  0.28858304 0.3424843  0.30056345 0.40402985 0.3483925
 0.31748122 0.24644688 0.44241396 0.38136336 0.28567678 0.44705957
 0.38774297 0.41305768 0.3249862  0.36734205 0.36758855 0.3582934
 0.44289377 0.34552714 0.36856848 0.40823612 0.35484946 0.44464618
 0.398916   0.47927588 0.22163917 0.3461608  0.4054486  0.28198546
 0.38106388 0.44930744 0.42173606 0.36833555 0.34740037 0.465201
 0.37970915 0.20723727 0.2548557  0.25251812 0.38183248 0.3713618
 0.43911064 0.35810822 0.44917852 0.43112987 0.35330483 0.44708177
 0.39893723 0.40381676 0.32128078 0.35924757 0.44094867 0.42679015
 0.4300211  0.37023625 0.36457396 0.43536294 0.25343508 0.3387566
 0.31756788 0.3926012  0.30982217 0.31272882 0.26912242 0.44022256
 0.39591002 0.30098224 0.36318323 0.37398675 0.3880015  0.39987165
 0.3860845  0.38843304 0.3991845 ]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


In [18]:
for i in range(5000):
  p,log_q,log_p = model(x_train)
  nll = F.nll_loss(p,y_train)
  loss = nll + 1.0/x_train.size()[0]*(log_q - log_p) 
  acc = (p.detach().numpy().argmax(axis=1) == y_train.detach().numpy()).mean()
  print('Acc.')
  print(acc)
  loss.backward()
  opt.step()
  opt.zero_grad()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Streaming output truncated to the last 5000 lines.
Acc.
0.7556818181818182
Acc.
0.6931818181818182
Acc.
0.7784090909090909
Acc.
0.7954545454545454
Acc.
0.6988636363636364
Acc.
0.7613636363636364
Acc.
0.7670454545454546
Acc.
0.625
Acc.
0.8011363636363636
Acc.
0.75
Acc.
0.7954545454545454
Acc.
0.8068181818181818
Acc.
0.7386363636363636
Acc.
0.7897727272727273
Acc.
0.8579545454545454
Acc.
0.8579545454545454
Acc.
0.7897727272727273
Acc.
0.5568181818181818
Acc.
0.8295454545454546
Acc.
0.7954545454545454
Acc.
0.7329545454545454
Acc.
0.7954545454545454
Acc.
0.6988636363636364
Acc.
0.7670454545454546
Acc.
0.8295454545454546
Acc.
0.6704545454545454
Acc.
0.7613636363636364
Acc.
0.7840909090909091
Acc.
0.8522727272727273
Acc.
0.8011363636363636
Acc.
0.7215909090909091
Acc.
0.8181818181818182
Acc.
0.7784090909090909
Acc.
0.6534090909090909
Acc.
0.5965909090909091
Acc.
0.5568181818181818
Acc.
0.8238636363636364
Acc.
0.7556818181818182
Acc.
0.8522727272727273
Acc.
0.8181818181818182
Acc.
0.721590909

In [19]:
n_mc_samples = 20 
ps = []
for m in range(n_mc_samples):
    p_tmp, _, _ = model(x_validation)
    ps.append(p_tmp.detach().numpy())
ps = np.stack(ps)
p = ps.mean(axis=0)
p_var = ps.var(axis=0).sum(axis=-1)
acc = (p.argmax(axis=1) == y_validation.detach().numpy()).mean()
print('Acc.')
print(acc)
print('Var.')
print(p_var)

Acc.
0.8850574712643678
Var.
[1.29751846e-01 2.57394314e-01 2.21859992e-01 4.29588884e-01
 3.30924004e-01 7.23076165e-02 1.49409741e-01 4.02478904e-01
 1.21624907e-02 1.25867486e-01 3.66482705e-01 9.73459333e-02
 2.84803092e-01 1.20446309e-02 3.15969646e-01 3.54961082e-02
 4.19165194e-01 2.42042206e-02 2.71928823e-03 2.44023174e-01
 4.55903143e-01 1.14123579e-02 1.51999339e-01 2.47074172e-01
 8.72659832e-02 2.11874843e-01 1.21854777e-02 4.40306409e-04
 4.35364783e-01 3.84259745e-02 3.89606029e-01 4.77096997e-03
 5.12415427e-04 2.79940933e-01 5.02451733e-02 3.69463980e-01
 6.98138865e-06 2.27162222e-04 1.69382393e-01 5.51951304e-02
 4.04345959e-01 4.49375212e-01 4.29619968e-01 4.45693195e-01
 3.15140426e-01 3.45614664e-02 7.56054670e-02 4.54016745e-01
 1.72293723e-01 1.80922210e-01 1.40189618e-01 1.72520995e-01
 1.04036301e-01 3.88103910e-02 1.64326397e-04 1.08870566e-01
 1.78578407e-01 3.90078574e-01 6.41256869e-02 4.15113151e-01
 8.65592062e-02 4.52724040e-01 2.89562762e-01 4.04113680

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


In [72]:
model.prior.sample(shape=[2,3,4],n_samples=5).shape

torch.Size([5, 2, 3, 4, 1])

In [84]:
class discriminator(torch.nn.Module):
    def __init__(self, inputSize,hiddenSize):
        super(discriminator, self).__init__()
        self.hidden_1 = torch.nn.Linear(inputSize, hiddenSize)
        self.hidden_2 = torch.nn.Linear(hiddenSize, hiddenSize)
        self.linear = torch.nn.Linear(hiddenSize, 1)
    def forward(self, x):
        h1 = F.relu(self.hidden_1(x))
        h2 = F.relu(self.hidden_2(h1))
        out = torch.sigmoid(self.linear(h2))
        return out


In [85]:
class implicit_q(torch.nn.Module):
    def __init__(self, inputSize, hiddenSize, outputSize,latentSize):
        super(implicit_q, self).__init__()
        self.latent_size = latentSize
        self.latent = torch.distributions.Normal(0.0,1.0)
        self.hidden_1 = torch.nn.Linear(latentSize, hiddenSize)
        self.hidden_2 = torch.nn.Linear(hiddenSize, hiddenSize)
        self.linear = torch.nn.Linear(hiddenSize, inputSize*outputSize)
    def forward(self,n_samples):
        latent_sample = self.latent.rsample([n_samples,self.latent_size])
        h1 = F.relu(self.hidden_1(latent_sample))
        h2 = F.relu(self.hidden_2(h1))
        out = self.linear(h2)
        return out

In [86]:
class logistic_ivi(torch.nn.Module):
    def __init__(self, inputSize):
        super(logistic_ivi, self).__init__()
        PI = 0.5
        SIGMA_1 = torch.FloatTensor([math.exp(-0)])
        SIGMA_2 = torch.FloatTensor([math.exp(-6)])
        self.prior = ScaleMixtureGaussian(PI,SIGMA_1,SIGMA_2)
        self.weights = implicit_q(inputSize,20,2,10)
        self.input_size = inputSize
    def forward(self, x,stds=None):
        sample = self.weights(n_samples=1)
        out = F.softmax(F.linear(x,torch.reshape(sample,[2,self.input_size])))
        return out, sample


In [87]:
model = logistic_ivi(x_train_numpy.shape[1])
d = discriminator(x_train_numpy.shape[1]*2,20)
opt_m = torch.optim.Adam(params=model.weights.parameters(),lr=0.001)
opt_d = torch.optim.Adam(params=d.parameters(),lr=0.001)

In [88]:
n_mc_samples = 20 
ps = []
for m in range(n_mc_samples):
    p_tmp, _ = model(x_validation)
    ps.append(p_tmp.detach().numpy())
ps = np.stack(ps)
p = ps.mean(axis=0)
p_var = ps.var(axis=0).sum(axis=-1)
acc = (p.argmax(axis=1) == y_validation.detach().numpy()).mean()
print('Acc.')
print(acc)
print('Var.')
print(p_var)

Acc.
0.6206896551724138
Var.
[0.00558097 0.00672631 0.00599301 0.01048419 0.01218843 0.00104006
 0.00077261 0.01199293 0.0011636  0.00031696 0.00597831 0.00011586
 0.02625233 0.0018926  0.00628657 0.00762354 0.01537291 0.00604929
 0.00253286 0.00025707 0.0036525  0.04681315 0.00111531 0.02146201
 0.01913704 0.00455101 0.0027497  0.02082478 0.01425446 0.00695674
 0.01977315 0.00538791 0.05033243 0.005263   0.00725031 0.00486943
 0.00863063 0.0399358  0.00119768 0.00439745 0.0045039  0.06223623
 0.01233753 0.04700293 0.01467515 0.00252287 0.01414289 0.00521417
 0.01014329 0.00051024 0.00286139 0.00754409 0.0049067  0.01832896
 0.00257714 0.00427198 0.0984104  0.01836412 0.01016516 0.0164788
 0.02712553 0.01996503 0.00505553 0.00428261 0.01181013 0.01366801
 0.09797276 0.007921   0.00422938 0.03576834 0.00090192 0.00530936
 0.00199223 0.02767008 0.00177001 0.00634346 0.00131512 0.01851135
 0.00230584 0.00115519 0.01209671 0.01850609 0.0085391  0.00073227
 0.00487072 0.0055719  0.011636  ]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


In [89]:
n_d_samples = 50
zeros = torch.zeros([n_d_samples])
ones = torch.ones([n_d_samples])
y_d = torch.reshape(torch.cat([ones,zeros],0),[-1,1])
for i in range(2000):
  p_samples = torch.squeeze(model.prior.sample([model.input_size*2],n_d_samples))
  q_samples = model.weights(n_d_samples)
  d_samples = torch.cat([p_samples,q_samples],0)
  p_d = d(d_samples)
  loss_d = F.binary_cross_entropy(p_d,y_d)
  loss_d.backward()
  opt_d.step()
  opt_d.zero_grad()
  opt_m.zero_grad()

  p, sample = model(x_train)
  nll = F.nll_loss(p,y_train)
  p_d = d(torch.reshape(sample,[1,-1]))
  print('p_discriminator')
  print(p_d)
  print('Ratio')
  ratio = (1-p_d)/p_d
  print(ratio)
  loss = nll + 1.0/x_train.size()[0]*(torch.log(ratio)).sum() 
  acc = (p.detach().numpy().argmax(axis=1) == y_train.detach().numpy()).mean()
  print('Acc.')
  print(acc)
  loss.backward()
  opt_m.step()
  opt_m.zero_grad()
  opt_d.zero_grad()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


Streaming output truncated to the last 5000 lines.
Acc.
0.9431818181818182
p_discriminator
tensor([[0.0003]], grad_fn=<SigmoidBackward0>)
Ratio
tensor([[3931.7476]], grad_fn=<DivBackward0>)
Acc.
0.9375
p_discriminator
tensor([[1.3738e-05]], grad_fn=<SigmoidBackward0>)
Ratio
tensor([[72790.5312]], grad_fn=<DivBackward0>)
Acc.
0.9375
p_discriminator
tensor([[0.0009]], grad_fn=<SigmoidBackward0>)
Ratio
tensor([[1105.7118]], grad_fn=<DivBackward0>)
Acc.
0.9375
p_discriminator
tensor([[7.4768e-05]], grad_fn=<SigmoidBackward0>)
Ratio
tensor([[13373.6211]], grad_fn=<DivBackward0>)
Acc.
0.9431818181818182
p_discriminator
tensor([[0.0006]], grad_fn=<SigmoidBackward0>)
Ratio
tensor([[1541.0599]], grad_fn=<DivBackward0>)
Acc.
0.9375
p_discriminator
tensor([[0.0003]], grad_fn=<SigmoidBackward0>)
Ratio
tensor([[2919.3088]], grad_fn=<DivBackward0>)
Acc.
0.9431818181818182
p_discriminator
tensor([[0.0002]], grad_fn=<SigmoidBackward0>)
Ratio
tensor([[5853.2773]], grad_fn=<DivBackward0>)
Acc.
0.9431818

In [90]:
n_mc_samples = 20 
ps = []
for m in range(n_mc_samples):
    p_tmp, _ = model(x_validation)
    ps.append(p_tmp.detach().numpy())
ps = np.stack(ps)
p = ps.mean(axis=0)
p_var = ps.var(axis=0).sum(axis=-1)
acc = (p.argmax(axis=1) == y_validation.detach().numpy()).mean()
print('Acc.')
print(acc)
print('Var.')
print(p_var)

Acc.
0.8390804597701149
Var.
[1.83646474e-03 3.58269835e-06 1.75770273e-10 2.89164200e-05
 1.18025455e-05 6.66338019e-04 1.14578498e-03 9.92681635e-07
 6.65648375e-04 5.82143082e-04 3.80356088e-02 3.71908024e-02
 9.61847109e-05 1.46075094e-04 1.63582875e-03 1.54135961e-07
 5.78818726e-04 4.63222477e-06 4.56814596e-05 2.09414028e-03
 3.45488879e-05 3.36233500e-17 9.04365152e-05 1.17951864e-03
 4.42417586e-05 4.66526653e-05 7.67196325e-05 8.96000711e-06
 1.67270482e-05 7.59739123e-06 2.39392859e-04 1.29024465e-06
 1.45107886e-04 1.39689637e-05 1.03543662e-05 3.53276788e-04
 8.85043261e-10 1.12128189e-14 8.75849184e-03 1.30831599e-04
 3.03932745e-03 9.16273333e-03 1.97917198e-05 7.63563776e-07
 5.15360013e-03 1.74293757e-06 1.30247063e-04 4.65204958e-10
 1.25391884e-13 1.39679178e-03 5.04468990e-05 5.28317294e-04
 5.75646918e-05 1.10670589e-05 3.73117539e-14 3.50845454e-04
 1.10371548e-06 1.83638222e-05 1.70770001e-13 2.32630016e-04
 1.02496579e-05 2.82605797e-05 9.03709047e-03 1.89161401

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
